# Random Forest Classifier for Rothko art work
### Adaboosted version

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import sklearn
import pickle

# For AdaBoost
from sklearn.ensemble import AdaBoostClassifier

## Load the Rothko art metrics

In [2]:
data = pd.DataFrame.from_csv('../../data/data.csv')
data = data[['year','shannon_entropy','mean_color_r','luminance','contrast', 'contour']]
data.head()

/Users/acanois/anaconda2/envs/lab/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


,year,shannon_entropy,mean_color_r,luminance,contrast,contour
0,1946,6.768127,195.046332,181.738624,0.827169,0.000056
1,1953,6.937948,228.983463,161.723046,0.587802,0.000000
2,1944,6.519651,217.905564,214.564295,0.853499,0.003758
3,1951,6.451747,191.229599,184.080065,0.706196,0.000063
4,1949,5.738259,230.154804,224.799111,0.658131,0.000000


In [3]:
# bin the years
bins = [1935, 1940, 1947, 1950, 1968, 1971]
data['year_bin']=pd.cut(data['year'], bins)
data.head()

,year,shannon_entropy,mean_color_r,luminance,contrast,contour,year_bin
0,1946,6.768127,195.046332,181.738624,0.827169,0.000056,"(1940, 1947]"
1,1953,6.937948,228.983463,161.723046,0.587802,0.000000,"(1950, 1968]"
2,1944,6.519651,217.905564,214.564295,0.853499,0.003758,"(1940, 1947]"
3,1951,6.451747,191.229599,184.080065,0.706196,0.000063,"(1950, 1968]"
4,1949,5.738259,230.154804,224.799111,0.658131,0.000000,"(1947, 1950]"


In [4]:
# Split into train and test data
features = data[['shannon_entropy','mean_color_r','luminance','contrast', 'contour']]
target = data['year_bin'].astype(str)
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=41)

In [5]:
features.columns.tolist()

['shannon_entropy', 'mean_color_r', 'luminance', 'contrast', 'contour']

## Create the Random Forest model and train it

In [6]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(features_train, target_train)
rf.score(features_train, target_train)

1.0

In [7]:
# Create the Adabooster - real numbers
ada_real = AdaBoostClassifier(
        base_estimator=rf,
    learning_rate=1,
    n_estimators=4000,
    algorithm='SAMME.R'
)
ada_real.fit(features_train, target_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          learning_rate=1, n_estimators=4000, random_state=None)

In [45]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.28051883, 0.1908065 , 0.19000446, 0.17473661, 0.1639336 ])

In [46]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, features.columns.tolist()), reverse=True)

[(0.28051882772466064, 'shannon_entropy'),
 (0.19080650301747532, 'mean_color_r'),
 (0.19000445833529497, 'luminance'),
 (0.17473661214568456, 'contrast'),
 (0.16393359877688443, 'contour')]

## Predict using the model

In [47]:
# predict the test data
predicted = rf.predict(features_test)

In [8]:
# Adaboost predict
ada_predicted = ada_real.predict(features_test)

In [48]:
# score on the test data
rf.score(features_test, target_test)

0.8333333333333334

In [10]:
# Adaboost score
ada_real.score(features_test, target_test)

0.8333333333333334

In [49]:
# score on the test
sklearn.metrics.accuracy_score(target_test,predicted)

0.8333333333333334

In [50]:
# Save the model to a file
filename = "RothkoRandomForestModelAdaBoost.pkl"
pickle.dump(rf, open(filename, "wb"))

In [51]:
# load the model from disk
loaded_rf = pickle.load(open(filename, "rb"))

In [52]:
loaded_rf.score(features_test, target_test)

0.8333333333333334